In [10]:
import tensorflow as tf
import numpy as np
import random

# modèle RNN
class RNNModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(RNNModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.rnn = tf.keras.layers.SimpleRNN(hidden_size)
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.rnn(x)
        return self.fc(x)

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", ["A)Tri par insertion", "B)Tri fusion", "C)Tri rapide", "D)Tri à bulles"], "C)Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", ["A) O(n log n)", "B) O(n^2)", "C) O(n)", "D) O(log n)"], "A) O(n log n)"),
]

# Encoder les données en indices
word_to_ix = {}
for question, options, _ in data:
    for word in question.split():
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
    for option in options:
        if option not in word_to_ix:
            word_to_ix[option] = len(word_to_ix)

# Taille du vocabulaire
vocab_size = len(word_to_ix)

embedding_dim = 10

model = RNNModel(vocab_size, embedding_dim, 10)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

for epoch in range(100):
    random.shuffle(data)
    for question, options, answer in data:
        model.reset_states()
        input_tensor = tf.constant([word_to_ix[word] for word in question.split()], dtype=tf.int32)
        input_tensor = tf.expand_dims(input_tensor, axis=0) 
        target_tensor = tf.constant([word_to_ix[answer]], dtype=tf.int32) 
        with tf.GradientTape() as tape:
            output = model(input_tensor)
            loss = loss_fn(target_tensor, output)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

def generate_exam_question(question, options, answer):
    random.shuffle(options)
    exam_question = f"{question} {' '.join(options)} La réponse correcte est {answer}."
    return exam_question

random_question, random_options, random_answer = random.choice(data)

exam_question = generate_exam_question(random_question, random_options, random_answer)

print(exam_question)


Quelle est la complexité temporelle du tri fusion ? D) O(log n) C) O(n) A) O(n log n) B) O(n^2) La réponse correcte est A) O(n log n).


In [1]:
import tensorflow as tf
import numpy as np
import random


class RNNModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(RNNModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.rnn = tf.keras.layers.SimpleRNN(hidden_size)
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.expand_dims(x, axis=0)
        x = self.rnn(x)
        return self.fc(x)

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", ["A)Tri par insertion", "B)Tri fusion", "C)Tri rapide", "D)Tri à bulles"], "C)Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", ["A) O(n log n)", "B) O(n^2)", "C) O(n)", "D) O(log n)"], "A) O(n log n)"),
]

# Encoder les données en indices
word_to_ix = {}
for question, options, _ in data:
    for word in question.split():
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
    for option in options:
        if option not in word_to_ix:
            word_to_ix[option] = len(word_to_ix)

# Taille du vocabulaire
vocab_size = len(word_to_ix)

embedding_dim = 10

model = RNNModel(vocab_size, embedding_dim, 10)

train_questions = [tf.constant([word_to_ix[word] for word in question.split()], dtype=tf.int32) for question, _, _ in data]
train_answers = [tf.constant([word_to_ix[answer]], dtype=tf.int32) for _, _, answer in data]

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

epochs = 100
for epoch in range(epochs):
    epoch_loss = 0
    train_accuracy.reset_states() 
    for input_tensor, target_tensor in zip(train_questions, train_answers):
        with tf.GradientTape() as tape:
            predictions = model(input_tensor)
            loss = loss_fn(target_tensor, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        epoch_loss += loss
        train_accuracy.update_state(target_tensor, predictions)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss.numpy():.4f}, Accuracy: {train_accuracy.result().numpy():.4f}")

model.save_weights("model_weights.h5")

def generate_exam_question(model, word_to_ix, data):
    random_question, random_options, random_answer = random.choice(data)
    input_tensor = tf.constant([word_to_ix[word] for word in random_question.split()], dtype=tf.int32)
    predictions = model(input_tensor)
    predicted_answer_index = tf.argmax(predictions, axis=-1).numpy()[0]
    predicted_answer = list(word_to_ix.keys())[predicted_answer_index]
    
    shuffled_options = random_options[:]
    random.shuffle(shuffled_options)

    return random_question, shuffled_options, predicted_answer

model.load_weights("model_weights.h5")

random_question, random_options, predicted_answer = generate_exam_question(model, word_to_ix, data)
exam_question = f"{random_question} {' '.join(random_options)} La réponse correcte est {predicted_answer}."

print(exam_question)


Epoch 1/100, Loss: 6.2058, Accuracy: 0.5000
Epoch 2/100, Loss: 6.1328, Accuracy: 0.5000
Epoch 3/100, Loss: 6.0677, Accuracy: 0.5000
Epoch 4/100, Loss: 6.0042, Accuracy: 0.5000
Epoch 5/100, Loss: 5.9414, Accuracy: 1.0000
Epoch 6/100, Loss: 5.8791, Accuracy: 1.0000
Epoch 7/100, Loss: 5.8170, Accuracy: 1.0000
Epoch 8/100, Loss: 5.7549, Accuracy: 1.0000
Epoch 9/100, Loss: 5.6927, Accuracy: 1.0000
Epoch 10/100, Loss: 5.6302, Accuracy: 1.0000
Epoch 11/100, Loss: 5.5672, Accuracy: 1.0000
Epoch 12/100, Loss: 5.5035, Accuracy: 1.0000
Epoch 13/100, Loss: 5.4391, Accuracy: 1.0000
Epoch 14/100, Loss: 5.3736, Accuracy: 1.0000
Epoch 15/100, Loss: 5.3071, Accuracy: 1.0000
Epoch 16/100, Loss: 5.2395, Accuracy: 1.0000
Epoch 17/100, Loss: 5.1705, Accuracy: 1.0000
Epoch 18/100, Loss: 5.1003, Accuracy: 1.0000
Epoch 19/100, Loss: 5.0287, Accuracy: 1.0000
Epoch 20/100, Loss: 4.9558, Accuracy: 1.0000
Epoch 21/100, Loss: 4.8815, Accuracy: 1.0000
Epoch 22/100, Loss: 4.8059, Accuracy: 1.0000
Epoch 23/100, Loss:

In [71]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quelle est la complexité temporelle de la recherche séquentielle ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n)"),
    ("Quelle est la complexité spatiale de l'algorithme de tri rapide ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Dijkstra"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche linéaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour trouver un arbre couvrant minimal dans un graphe pondéré ?", 
     ["Algorithme de Prim", "Algorithme de Kruskal", "Algorithme de Dijkstra", "Algorithme de Bellman-Ford"], 
     "Algorithme de Prim"),
    ("Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe avec des arêtes de poids négatif ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Bellman-Ford"),
    ("Quelle est la complexité temporelle de l'algorithme de Floyd-Warshall ?", 
     ["O(n^3)", "O(n^2)", "O(n log n)", "O(n)"], 
     "O(n^3)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 
     ["Parcours en profondeur d'abord (DFS)", "Parcours en largeur d'abord (BFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en profondeur d'abord (DFS)"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche en profondeur ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en largeur dans un graphe ?", 
     ["Parcours en largeur d'abord (BFS)", "Parcours en profondeur d'abord (DFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en largeur d'abord (BFS)")
]

def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(processed_data[0][1]))))
model.add(Dense(len(processed_data[0][1]), activation='softmax'))  
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01)) 

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)

def generate_exam(model):
    exam = ""
    for i, (question, options, correct_answer) in enumerate(processed_data, start=1):  
        exam += f"Question {i}:\n{question}\n"  
        for j, option in enumerate(options, start=1):
            exam += f"{chr(65 + j)}) {option}\n"  
        exam += f"Réponse correcte: {chr(65 + np.argmax(correct_answer))}\n\n"  # Affichage de la réponse correcte
    return exam

print(generate_exam(model))


Epoch 1/4
1/1 [==============================] - 3s 3s/step - loss: 1.3733
Epoch 2/4
1/1 [==============================] - 0s 14ms/step - loss: 1.0154
Epoch 3/4
1/1 [==============================] - 0s 24ms/step - loss: 0.8618
Epoch 4/4
1/1 [==============================] - 0s 16ms/step - loss: 1.1046
Question 1:
Quel est l'algorithme de tri le plus rapide ?
B) [1. 0. 0. 0.]
C) [0. 1. 0. 0.]
D) [0. 0. 1. 0.]
E) [0. 0. 0. 1.]
Réponse correcte: C

Question 2:
Quelle est la complexité temporelle du tri fusion ?
B) [1. 0. 0. 0.]
C) [0. 1. 0. 0.]
D) [0. 0. 1. 0.]
E) [0. 0. 0. 1.]
Réponse correcte: A

Question 3:
Quel est l'algorithme de recherche le plus efficace ?
B) [1. 0. 0. 0.]
C) [0. 1. 0. 0.]
D) [0. 0. 1. 0.]
E) [0. 0. 0. 1.]
Réponse correcte: B

Question 4:
Quelle est la complexité temporelle de la recherche binaire ?
B) [1. 0. 0. 0.]
C) [0. 1. 0. 0.]
D) [0. 0. 1. 0.]
E) [0. 0. 0. 1.]
Réponse correcte: A

Question 5:
Quel est l'algorithme de tri adapté aux listes presque triées ?


In [77]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quelle est la complexité temporelle de la recherche séquentielle ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n)"),
    ("Quelle est la complexité spatiale de l'algorithme de tri rapide ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Dijkstra"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche linéaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour trouver un arbre couvrant minimal dans un graphe pondéré ?", 
     ["Algorithme de Prim", "Algorithme de Kruskal", "Algorithme de Dijkstra", "Algorithme de Bellman-Ford"], 
     "Algorithme de Prim"),
    ("Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe avec des arêtes de poids négatif ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Bellman-Ford"),
    ("Quelle est la complexité temporelle de l'algorithme de Floyd-Warshall ?", 
     ["O(n^3)", "O(n^2)", "O(n log n)", "O(n)"], 
     "O(n^3)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 
     ["Parcours en profondeur d'abord (DFS)", "Parcours en largeur d'abord (BFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en profondeur d'abord (DFS)"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche en profondeur ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en largeur dans un graphe ?", 
     ["Parcours en largeur d'abord (BFS)", "Parcours en profondeur d'abord (DFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en largeur d'abord (BFS)")
]

def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(processed_data[0][1]))))  # Input shape: (None, 4) pour les options
model.add(Dense(len(processed_data[0][1]), activation='softmax'))  # 4 pour le nombre d'options
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))  # Modifier 'lr' en 'learning_rate'

# Entraînement du modèle
x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)


def generate_exam(model):
    exam = ""
    for i, (question, options, correct_answer) in enumerate(processed_data, start=1):  
        exam += f"Question {i}:\n{question}\n"  
        for j, option in enumerate(options, start=1):
            option_index = np.argmax(option)
            option_char = chr(65 + option_index)
            exam += f"{option_char}) {data[i-1][1][option_index]}\n"  
        correct_answer_index = np.argmax(correct_answer)
        correct_answer_char = chr(65 + correct_answer_index)
        exam += f"Réponse correcte: {correct_answer_char}\n\n"  
    return exam

print(generate_exam(model))




Epoch 1/4
1/1 [==============================] - 4s 4s/step - loss: 1.4151
Epoch 2/4
1/1 [==============================] - 0s 11ms/step - loss: 1.0526
Epoch 3/4
1/1 [==============================] - 0s 16ms/step - loss: 0.8722
Epoch 4/4
1/1 [==============================] - 0s 16ms/step - loss: 1.1387
Question 1:
Quel est l'algorithme de tri le plus rapide ?
A) Tri par insertion
B) Tri fusion
C) Tri rapide
D) Tri à bulles
Réponse correcte: C

Question 2:
Quelle est la complexité temporelle du tri fusion ?
A) O(n log n)
B) O(n^2)
C) O(n)
D) O(log n)
Réponse correcte: A

Question 3:
Quel est l'algorithme de recherche le plus efficace ?
A) Recherche séquentielle
B) Recherche binaire
C) Recherche exponentielle
D) Recherche linéaire
Réponse correcte: B

Question 4:
Quelle est la complexité temporelle de la recherche binaire ?
A) O(log n)
B) O(n)
C) O(n log n)
D) O(n^2)
Réponse correcte: A

Question 5:
Quel est l'algorithme de tri adapté aux listes presque triées ?
A) Tri par insertion
B)

In [81]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LSTM, Dense

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
]

def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential([
    LSTM(128, input_shape=(None, len(processed_data[0][1]))),
    Dense(len(processed_data[0][1]), activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)

def generate_question(model):
    random_index = random.randint(0, len(processed_data) - 1)
    question, options_one_hot, correct_answer_one_hot = processed_data[random_index]
    
    predictions = []

    for option_one_hot in options_one_hot:
        option_one_hot = np.expand_dims(option_one_hot, axis=0)  
        option_one_hot = np.expand_dims(option_one_hot, axis=1)
        
        # Faire la prédiction et ajouter le résultat à la liste des prédictions
        prediction = model.predict(option_one_hot)
        predictions.append(prediction)

    correct_answer_index = np.argmax(correct_answer_one_hot)
    
    return question, predictions, correct_answer_index

question, options, correct_answer_index = generate_question(model)
print("Question:", question)
print("Options:", options)
print("Réponse correcte:", correct_answer_index)


Epoch 1/4
1/1 [==============================] - 4s 4s/step - loss: 1.4016
Epoch 2/4
1/1 [==============================] - 0s 0s/step - loss: 1.0549
Epoch 3/4
1/1 [==============================] - 0s 8ms/step - loss: 1.1182
Epoch 4/4
1/1 [==============================] - 0s 29ms/step
Question: Quelle est la complexité temporelle du tri fusion ?
Options: [array([[0.31464076, 0.23870973, 0.24110721, 0.20554236]], dtype=float32), array([[0.32907745, 0.24333796, 0.23483215, 0.1927524 ]], dtype=float32), array([[0.3408757 , 0.2337325 , 0.23884836, 0.18654343]], dtype=float32), array([[0.35642144, 0.2350155 , 0.22970593, 0.17885712]], dtype=float32)]
Réponse correcte: 0


In [85]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LSTM, Dense

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
]


def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential([
    LSTM(128, input_shape=(None, len(processed_data[0][1]))),
    Dense(len(processed_data[0][1]), activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)


def generate_exam(model, num_questions):
    available_indices = list(range(len(processed_data)))  
    exam_questions = []
    
    for _ in range(num_questions):
        if not available_indices:
            break  
        
        random_index = random.choice(available_indices) 
        question, options_one_hot, correct_answer_one_hot = processed_data[random_index]
        
        available_indices.remove(random_index)

        # Créer une liste pour stocker les prédictions
        predictions = []

        # Faire une prédiction pour chaque option
        for option_one_hot in options_one_hot:
            option_one_hot = np.expand_dims(option_one_hot, axis=0)  
            option_one_hot = np.expand_dims(option_one_hot, axis=1)

            prediction = model.predict(option_one_hot)
            predictions.append(prediction)

        correct_answer_index = np.argmax(correct_answer_one_hot)

        exam_questions.append((question, predictions, correct_answer_index))
    
    return exam_questions


exam_questions = generate_exam(model, num_questions=5)
for i, (question, options, correct_answer_index) in enumerate(exam_questions, start=1):
    print(f"Question {i}: {question}")
    print("Options:", options)
    print("Réponse correcte:", correct_answer_index)
    print()


Epoch 1/4
1/1 [==============================] - 2s 2s/step - loss: 1.3725
Epoch 2/4
1/1 [==============================] - 0s 0s/step - loss: 1.0202
Epoch 3/4
1/1 [==============================] - 0s 14ms/step - loss: 1.1021
Epoch 4/4
1/1 [==============================] - 0s 37ms/step
Question 1: Quelle est la complexité temporelle du tri fusion ?
Options: [array([[0.31275326, 0.2472328 , 0.23692445, 0.20308952]], dtype=float32), array([[0.31857815, 0.24148715, 0.23828574, 0.201649  ]], dtype=float32), array([[0.34096515, 0.23874305, 0.24113461, 0.17915718]], dtype=float32), array([[0.3666095 , 0.23423356, 0.23054193, 0.1686149 ]], dtype=float32)]
Réponse correcte: 0

Question 2: Quel est l'algorithme de tri adapté aux listes presque triées ?
Options: [array([[0.31275326, 0.2472328 , 0.23692445, 0.20308952]], dtype=float32), array([[0.31857815, 0.24148715, 0.23828574, 0.201649  ]], dtype=float32), array([[0.34096515, 0.23874305, 0.24113461, 0.17915718]], dtype=float32), array([[0.36

In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quelle est la complexité temporelle de la recherche séquentielle ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n)"),
    ("Quelle est la complexité spatiale de l'algorithme de tri rapide ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Dijkstra"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche linéaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour trouver un arbre couvrant minimal dans un graphe pondéré ?", 
     ["Algorithme de Prim", "Algorithme de Kruskal", "Algorithme de Dijkstra", "Algorithme de Bellman-Ford"], 
     "Algorithme de Prim"),
    ("Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe avec des arêtes de poids négatif ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Bellman-Ford"),
    ("Quelle est la complexité temporelle de l'algorithme de Floyd-Warshall ?", 
     ["O(n^3)", "O(n^2)", "O(n log n)", "O(n)"], 
     "O(n^3)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 
     ["Parcours en profondeur d'abord (DFS)", "Parcours en largeur d'abord (BFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en profondeur d'abord (DFS)"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche en profondeur ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en largeur dans un graphe ?", 
     ["Parcours en largeur d'abord (BFS)", "Parcours en profondeur d'abord (DFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en largeur d'abord (BFS)")
]


def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(processed_data[0][1]))))  
model.add(Dense(len(processed_data[0][1]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)

def generate_question(model):
    
    question_idx = random.randint(0, len(data) - 1)
    question_text, options, correct_answer = processed_data[question_idx]
    
    predicted_answer = model.predict(np.array([options]))
    predicted_answer_index = np.argmax(predicted_answer)
    predicted_answer_text = data[question_idx][1][predicted_answer_index]
    
    print("Question:", question_text)
    for i, option in enumerate(options):
        print(f"{chr(65 + i)}) {data[question_idx][1][i]}")
    
    print("Réponse prédite:", predicted_answer_text)
    print("Réponse correcte:", correct_answer)
    print()

for _ in range(10):
    generate_question(model)


Epoch 1/4
1/1 [==============================] - 5s 5s/step - loss: 1.3554
Epoch 2/4
1/1 [==============================] - 0s 49ms/step - loss: 0.9705
Epoch 3/4
1/1 [==============================] - 0s 32ms/step - loss: 0.9567
Epoch 4/4
1/1 [==============================] - 1s 983ms/step
Question: Quelle est la complexité temporelle du tri fusion ?
A) O(n log n)
B) O(n^2)
C) O(n)
D) O(log n)
Réponse prédite: O(n log n)
Réponse correcte: [1. 0. 0. 0.]

1/1 [==============================] - 0s 17ms/step
Question: Quelle est la complexité temporelle de la recherche binaire ?
A) O(log n)
B) O(n)
C) O(n log n)
D) O(n^2)
Réponse prédite: O(log n)
Réponse correcte: [1. 0. 0. 0.]

1/1 [==============================] - 0s 25ms/step
Question: Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?
A) Algorithme de Dijkstra
B) Algorithme de Bellman-Ford
C) Algorithme de Floyd-Warshall
D) Algorithme de Kruskal
Réponse prédite: Algorithme de Dijkstra
Réponse co

In [8]:
import random
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM


data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quelle est la complexité temporelle de la recherche séquentielle ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n)"),
    ("Quelle est la complexité spatiale de l'algorithme de tri rapide ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Dijkstra"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche linéaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour trouver un arbre couvrant minimal dans un graphe pondéré ?", 
     ["Algorithme de Prim", "Algorithme de Kruskal", "Algorithme de Dijkstra", "Algorithme de Bellman-Ford"], 
     "Algorithme de Prim"),
    ("Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe avec des arêtes de poids négatif ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Bellman-Ford"),
    ("Quelle est la complexité temporelle de l'algorithme de Floyd-Warshall ?", 
     ["O(n^3)", "O(n^2)", "O(n log n)", "O(n)"], 
     "O(n^3)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 
     ["Parcours en profondeur d'abord (DFS)", "Parcours en largeur d'abord (BFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en profondeur d'abord (DFS)"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche en profondeur ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en largeur dans un graphe ?", 
     ["Parcours en largeur d'abord (BFS)", "Parcours en profondeur d'abord (DFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en largeur d'abord (BFS)")
]

def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(processed_data[0][1])))) 
model.add(Dense(len(processed_data[0][1]), activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01)) 

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)

def generate_question(model):
    # question aléatoire
    question_idx = random.randint(0, len(data) - 1)
    question_text, options, correct_answer = processed_data[question_idx]
    
    # Prédire la réponse
    predicted_answer = model.predict(np.array([options]))
    predicted_answer_index = np.argmax(predicted_answer)
    predicted_answer_text = data[question_idx][1][predicted_answer_index]
    
    # Pas besoin de décoder les options et la réponse correcte
    options_text = [str(i) for i in range(len(options))]
    correct_answer_text = str(np.argmax(correct_answer))
    
    return {"question": question_text, "options": options_text, "correct_answer": correct_answer_text}

# je Génére et afficher 10 QCM
generated_questions = [generate_question(model) for _ in range(10)]

# le fichier XML pour Moodle creation brute
root = ET.Element("quiz")
for i, qcm in enumerate(generated_questions, start=1):
    question = ET.SubElement(root, "question", type="multichoice")
    name = ET.SubElement(question, "name")
    text = ET.SubElement(name, "text")
    text.text = f"Question {i}"
    questiontext = ET.SubElement(question, "questiontext", format="html")
    text = ET.SubElement(questiontext, "text")
    text.text = f"<![CDATA[<p></p><div><div><div>{qcm['question']}</div></div><br></div><br><p></p>]]>"
    generalfeedback = ET.SubElement(question, "generalfeedback", format="html")
    text = ET.SubElement(generalfeedback, "text")
    text.text = "<![CDATA[<p>none</p>]]>"
    defaultgrade = ET.SubElement(question, "defaultgrade")
    defaultgrade.text = "1.0000000"
    penalty = ET.SubElement(question, "penalty")
    penalty.text = "0.3333333"
    hidden = ET.SubElement(question, "hidden")
    hidden.text = "0"
    single = ET.SubElement(question, "single")
    single.text = "true"
    shuffleanswers = ET.SubElement(question, "shuffleanswers")
    shuffleanswers.text = "true"
    answernumbering = ET.SubElement(question, "answernumbering")
    answernumbering.text = "abc"
    for j, option in enumerate(qcm["options"], start=1):
        answer = ET.SubElement(question, "answer", fraction="100" if option == qcm["correct_answer"] else "0", format="html")
        text = ET.SubElement(answer, "text")
        text.text = f"<![CDATA[<p></p><div><div><div>{option}</div></div><br></div><br><p></p>]]>"
        feedback = ET.SubElement(answer, "feedback", format="html")
        text = ET.SubElement(feedback, "text")
        text.text = "<![CDATA[<p>none</p>]]>"

tree = ET.ElementTree(root)
tree.write("BANK_QUESTIONS_1.xml", encoding="utf-8", xml_declaration=True)


Epoch 1/4
1/1 [==============================] - 3s 3s/step - loss: 1.3814
Epoch 2/4
1/1 [==============================] - 0s 25ms/step - loss: 1.0316
Epoch 3/4
1/1 [==============================] - 0s 20ms/step - loss: 0.9085
Epoch 4/4
1/1 [==============================] - 0s 28ms/step


In [26]:
import random
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quelle est la complexité temporelle de la recherche séquentielle ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n)"),
    ("Quelle est la complexité spatiale de l'algorithme de tri rapide ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Dijkstra"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche linéaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour trouver un arbre couvrant minimal dans un graphe pondéré ?", 
     ["Algorithme de Prim", "Algorithme de Kruskal", "Algorithme de Dijkstra", "Algorithme de Bellman-Ford"], 
     "Algorithme de Prim"),
    ("Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe avec des arêtes de poids négatif ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Bellman-Ford"),
    ("Quelle est la complexité temporelle de l'algorithme de Floyd-Warshall ?", 
     ["O(n^3)", "O(n^2)", "O(n log n)", "O(n)"], 
     "O(n^3)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 
     ["Parcours en profondeur d'abord (DFS)", "Parcours en largeur d'abord (BFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en profondeur d'abord (DFS)"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche en profondeur ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en largeur dans un graphe ?", 
     ["Parcours en largeur d'abord (BFS)", "Parcours en profondeur d'abord (DFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en largeur d'abord (BFS)")
]

def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(processed_data[0][1])))) 
model.add(Dense(len(processed_data[0][1]), activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01)) 

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)

def generate_question(model, generated_questions):
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions]
    question_idx = random.choice(unused_questions)
    
    generated_questions.add(question_idx)
    question_text, options, correct_answer = processed_data[question_idx]
    
    predicted_answer = model.predict(np.array([options]))
    predicted_answer_index = np.argmax(predicted_answer)
    predicted_answer_text = options[predicted_answer_index]
   
    options_text = [str(i) for i in range(len(options))]
    correct_answer_text = str(np.argmax(correct_answer))
    
    return {"question": question_text, "options": options_text, "correct_answer": correct_answer_text}

# je genere et afficher 10 QCM
generated_questions = set()  # Pour garder une trace des questions déjà générées
for _ in range(10):
    question = generate_question(model, generated_questions)
    print(question)


# je Génére le fichier XML pour Moodle
root = ET.Element("quiz")
for i, question_idx in enumerate(generated_questions, start=1):
    qcm = processed_data[question_idx]
    question = ET.SubElement(root, "question", type="multichoice")
    name = ET.SubElement(question, "name")
    text = ET.SubElement(name, "text")
    text.text = f"Question {i}"
    questiontext = ET.SubElement(question, "questiontext", format="html")
    text = ET.SubElement(questiontext, "text")
    text.text = f"<![CDATA[<p></p><div><div><div>{qcm[0]}</div></div><br></div><br><p></p>]]>"  # Utiliser qcm[0] pour la question
    generalfeedback = ET.SubElement(question, "generalfeedback", format="html")
    text = ET.SubElement(generalfeedback, "text")
    text.text = "<![CDATA[<p>none</p>]]>"
    defaultgrade = ET.SubElement(question, "defaultgrade")
    defaultgrade.text = "1.0000000"
    penalty = ET.SubElement(question, "penalty")
    penalty.text = "0.3333333"
    hidden = ET.SubElement(question, "hidden")
    hidden.text = "0"
    single = ET.SubElement(question, "single")
    single.text = "true"
    shuffleanswers = ET.SubElement(question, "shuffleanswers")
    shuffleanswers.text = "true"
    answernumbering = ET.SubElement(question, "answernumbering")
    answernumbering.text = "abc"
    for j, option in enumerate(qcm[1], start=1): 
        answer = ET.SubElement(question, "answer", fraction="100" if np.argmax(qcm[2]) == j - 1 else "0", format="html")
        text = ET.SubElement(answer, "text")
        text.text = f"<![CDATA[<p></p><div><div><div>{option}</div></div><br></div><br><p></p>]]>"
        feedback = ET.SubElement(answer, "feedback", format="html")
        text = ET.SubElement(feedback, "text")
        text.text = "<![CDATA[<p>none</p>]]>"

tree = ET.ElementTree(root)
tree.write("BANK_QUESTIONS_2.xml", encoding="utf-8", xml_declaration=True)


Epoch 1/4
1/1 [==============================] - 3s 3s/step - loss: 1.3828
Epoch 2/4
1/1 [==============================] - 0s 15ms/step - loss: 1.0318
Epoch 3/4
1/1 [==============================] - 0s 29ms/step - loss: 0.8918
Epoch 4/4
1/1 [==============================] - 1s 537ms/step
{'question': "Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 'options': ['0', '1', '2', '3'], 'correct_answer': '0'}
1/1 [==============================] - 0s 29ms/step
{'question': "Quel est l'algorithme de tri adapté aux listes de petite taille ?", 'options': ['0', '1', '2', '3'], 'correct_answer': '0'}
1/1 [==============================] - 0s 53ms/step
{'question': "Quel est l'algorithme utilisé pour trouver un arbre couvrant minimal dans un graphe pondéré ?", 'options': ['0', '1', '2', '3'], 'correct_answer': '0'}
1/1 [==============================] - 0s 29ms/step
{'question': "Quelle est la complexité spatiale de l'algorithme de tri rapide ?", 'opt

In [30]:
import random
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quelle est la complexité temporelle de la recherche séquentielle ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n)"),
    ("Quelle est la complexité spatiale de l'algorithme de tri rapide ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Dijkstra"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche linéaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour trouver un arbre couvrant minimal dans un graphe pondéré ?", 
     ["Algorithme de Prim", "Algorithme de Kruskal", "Algorithme de Dijkstra", "Algorithme de Bellman-Ford"], 
     "Algorithme de Prim"),
    ("Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe avec des arêtes de poids négatif ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Bellman-Ford"),
    ("Quelle est la complexité temporelle de l'algorithme de Floyd-Warshall ?", 
     ["O(n^3)", "O(n^2)", "O(n log n)", "O(n)"], 
     "O(n^3)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 
     ["Parcours en profondeur d'abord (DFS)", "Parcours en largeur d'abord (BFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en profondeur d'abord (DFS)"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche en profondeur ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en largeur dans un graphe ?", 
     ["Parcours en largeur d'abord (BFS)", "Parcours en profondeur d'abord (DFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en largeur d'abord (BFS)")
]

def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(processed_data[0][1])))) 
model.add(Dense(len(processed_data[0][1]), activation='softmax'))  
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01)) 

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)

def generate_question(model, generated_questions):
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions]
    question_idx = random.choice(unused_questions)
    generated_questions.add(question_idx)
    question_text, options, correct_answer = processed_data[question_idx]
    
    # Prédire la réponse
    predicted_answer = model.predict(np.array([options]))
    predicted_answer_index = np.argmax(predicted_answer)
    predicted_answer_text = options[predicted_answer_index]
    
    # Convertir les options en textes
    options_text = [option for option in data[question_idx][1]]
    correct_answer_text = data[question_idx][2]
    
    return {"question": question_text, "options": options_text, "correct_answer": correct_answer_text}

generated_questions = set()  
for _ in range(10):
    question = generate_question(model, generated_questions)
    print(question)

# Générer le fichier XML pour Moodle
root = ET.Element("quiz")
for i, question_idx in enumerate(generated_questions, start=1):
    qcm = processed_data[question_idx] 
    question = ET.SubElement(root, "question", type="multichoice")
    name = ET.SubElement(question, "name")
    text = ET.SubElement(name, "text")
    text.text = f"Question {i}"
    questiontext = ET.SubElement(question, "questiontext", format="html")
    text = ET.SubElement(questiontext, "text")
    text.text = f"<![CDATA[<p></p><div><div><div>{qcm[0]}</div></div><br></div><br><p></p>]]>"
    generalfeedback = ET.SubElement(question, "generalfeedback", format="html")
    text = ET.SubElement(generalfeedback, "text")
    text.text = "<![CDATA[<p>none</p>]]>"
    defaultgrade = ET.SubElement(question, "defaultgrade")
    defaultgrade.text = "1.0000000"
    penalty = ET.SubElement(question, "penalty")
    penalty.text = "0.3333333"
    hidden = ET.SubElement(question, "hidden")
    hidden.text = "0"
    single = ET.SubElement(question, "single")
    single.text = "true"
    shuffleanswers = ET.SubElement(question, "shuffleanswers")
    shuffleanswers.text = "true"
    answernumbering = ET.SubElement(question, "answernumbering")
    answernumbering.text = "abc"
    for j, option in enumerate(qcm[1], start=1): 
        answer = ET.SubElement(question, "answer", fraction="100" if np.argmax(qcm[2]) == j - 1 else "0", format="html")
        text = ET.SubElement(answer, "text")
        text.text = f"<![CDATA[<p></p><div><div><div>{option}</div></div><br></div><br><p></p>]]>"
        feedback = ET.SubElement(answer, "feedback", format="html")
        text = ET.SubElement(feedback, "text")
        text.text = "<![CDATA[<p>none</p>]]>"

tree = ET.ElementTree(root)
tree.write("BANK_QUESTIONS_3.xml", encoding="utf-8", xml_declaration=True)


Epoch 1/4
1/1 [==============================] - 4s 4s/step - loss: 1.3747
Epoch 2/4
1/1 [==============================] - 0s 17ms/step - loss: 1.0307
Epoch 3/4
1/1 [==============================] - 0s 19ms/step - loss: 0.8956
Epoch 4/4
1/1 [==============================] - 1s 639ms/step
{'question': "Quel est l'algorithme de tri adapté aux listes presque triées ?", 'options': ['Tri par insertion', 'Tri fusion', 'Tri rapide', 'Tri à bulles'], 'correct_answer': 'Tri par insertion'}
1/1 [==============================] - 0s 25ms/step
{'question': 'Quelle est la complexité temporelle de la recherche binaire ?', 'options': ['O(log n)', 'O(n)', 'O(n log n)', 'O(n^2)'], 'correct_answer': 'O(log n)'}
1/1 [==============================] - 0s 30ms/step
{'question': "Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe avec des arêtes de poids négatif ?", 'options': ['Algorithme de Dijkstra', 'Algorithme de Bellman-Ford', 'Algorithme de Floyd-Warshall', 'Algorithme 

In [31]:
##########  Proposition en tableau Numpy  dans le fichier .xml ##########

import random
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quelle est la complexité temporelle de la recherche séquentielle ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n)"),
    ("Quelle est la complexité spatiale de l'algorithme de tri rapide ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Dijkstra"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche linéaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour trouver un arbre couvrant minimal dans un graphe pondéré ?", 
     ["Algorithme de Prim", "Algorithme de Kruskal", "Algorithme de Dijkstra", "Algorithme de Bellman-Ford"], 
     "Algorithme de Prim"),
    ("Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe avec des arêtes de poids négatif ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Bellman-Ford"),
    ("Quelle est la complexité temporelle de l'algorithme de Floyd-Warshall ?", 
     ["O(n^3)", "O(n^2)", "O(n log n)", "O(n)"], 
     "O(n^3)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 
     ["Parcours en profondeur d'abord (DFS)", "Parcours en largeur d'abord (BFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en profondeur d'abord (DFS)"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche en profondeur ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en largeur dans un graphe ?", 
     ["Parcours en largeur d'abord (BFS)", "Parcours en profondeur d'abord (DFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en largeur d'abord (BFS)")
]


def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)


model = Sequential()
model.add(LSTM(128, input_shape=(None, len(processed_data[0][1]))))  
model.add(Dense(len(processed_data[0][1]), activation='softmax'))  
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01)) 

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)

def generate_question(model, generated_questions):
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions]
    question_idx = random.choice(unused_questions)
    generated_questions.add(question_idx)
    question_text, options, correct_answer = processed_data[question_idx]
    
    predicted_answer = model.predict(np.array([options]))
    predicted_answer_index = np.argmax(predicted_answer)
    predicted_answer_text = options[predicted_answer_index]
    
   
    options_text = [option for option in data[question_idx][1]]
    correct_answer_text = data[question_idx][2]
    
    return {"question": question_text, "options": options_text, "correct_answer": correct_answer_text}

generated_questions = set() 
for _ in range(10):
    question = generate_question(model, generated_questions)
    print(question)


with open("BANK_QUESTIONS_4.xml", "w") as f:
    f.write("<quiz>\n")
    for i, question_idx in enumerate(generated_questions, start=1):
        qcm = processed_data[question_idx]  
        f.write(f"  <!-- question: {i} -->\n")
        f.write(f"  <question type=\"multichoice\">\n")
        f.write(f"    <name>\n")
        f.write(f"      <text>Question {i}</text>\n")
        f.write(f"    </name>\n")
        f.write(f"    <questiontext format=\"html\">\n")
        f.write(f"      <text><![CDATA[<pre>{qcm[0]}</pre>]]></text>\n")
        f.write(f"    </questiontext>\n")
        f.write(f"    <generalfeedback format=\"html\">\n")
        f.write(f"      <text><![CDATA[<p>none</p>]]></text>\n")
        f.write(f"    </generalfeedback>\n")
        f.write(f"    <defaultgrade>1.0000000</defaultgrade>\n")
        f.write(f"    <penalty>0.3333333</penalty>\n")
        f.write(f"    <hidden>0</hidden>\n")
        f.write(f"    <single>true</single>\n")
        f.write(f"    <shuffleanswers>true</shuffleanswers>\n")
        f.write(f"    <answernumbering>abc</answernumbering>\n")
        for j, option in enumerate(qcm[1], start=1):
            f.write(f"    <answer fraction=\"{'100' if np.argmax(qcm[2]) == j - 1 else '0'}\" format=\"html\">\n")
            f.write(f"      <text><![CDATA[<pre>{option}</pre>]]></text>\n")
            f.write(f"      <feedback format=\"html\">\n")
            f.write(f"        <text><![CDATA[<p>none</p>]]></text>\n")
            f.write(f"      </feedback>\n")
            f.write(f"    </answer>\n")
        f.write(f"  </question>\n")
    f.write("</quiz>\n")


Epoch 1/4
1/1 [==============================] - 3s 3s/step - loss: 1.3891
Epoch 2/4
1/1 [==============================] - 0s 15ms/step - loss: 1.0130
Epoch 3/4
1/1 [==============================] - 0s 26ms/step - loss: 0.9066
Epoch 4/4
1/1 [==============================] - 1s 743ms/step
{'question': "Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?", 'options': ['Algorithme de Dijkstra', 'Algorithme de Bellman-Ford', 'Algorithme de Floyd-Warshall', 'Algorithme de Kruskal'], 'correct_answer': 'Algorithme de Dijkstra'}
1/1 [==============================] - 0s 35ms/step
{'question': "Quelle est la complexité temporelle de l'algorithme de Floyd-Warshall ?", 'options': ['O(n^3)', 'O(n^2)', 'O(n log n)', 'O(n)'], 'correct_answer': 'O(n^3)'}
1/1 [==============================] - 0s 64ms/step
{'question': "Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 'options': ['O(n log n)', 'O(n)', 'O(log n)', 'O(n^2)'], 'correct_answer': 'O

In [33]:
########## Proposition en texte dans le .xml et retour a la ligne pour chque nouvelle question ##########

import random
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quelle est la complexité temporelle de la recherche séquentielle ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n)"),
    ("Quelle est la complexité spatiale de l'algorithme de tri rapide ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Dijkstra"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche linéaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour trouver un arbre couvrant minimal dans un graphe pondéré ?", 
     ["Algorithme de Prim", "Algorithme de Kruskal", "Algorithme de Dijkstra", "Algorithme de Bellman-Ford"], 
     "Algorithme de Prim"),
    ("Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe avec des arêtes de poids négatif ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Bellman-Ford"),
    ("Quelle est la complexité temporelle de l'algorithme de Floyd-Warshall ?", 
     ["O(n^3)", "O(n^2)", "O(n log n)", "O(n)"], 
     "O(n^3)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 
     ["Parcours en profondeur d'abord (DFS)", "Parcours en largeur d'abord (BFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en profondeur d'abord (DFS)"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche en profondeur ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en largeur dans un graphe ?", 
     ["Parcours en largeur d'abord (BFS)", "Parcours en profondeur d'abord (DFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en largeur d'abord (BFS)")
]

def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(processed_data[0][1])))) 
model.add(Dense(len(processed_data[0][1]), activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)

def generate_question(model, generated_questions):
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions]
    question_idx = random.choice(unused_questions)
    generated_questions.add(question_idx)
    question_text, options, correct_answer = processed_data[question_idx]
    
    predicted_answer = model.predict(np.array([options]))
    predicted_answer_index = np.argmax(predicted_answer)
    predicted_answer_text = options[predicted_answer_index]
    
    options_text = [option for option in data[question_idx][1]]
    correct_answer_text = data[question_idx][2]
    
    return {"question": question_text, "options": options_text, "correct_answer": correct_answer_text}

generated_questions = set() 
for _ in range(10):
    question = generate_question(model, generated_questions)
    print(question)

with open("BANK_QUESTIONS_5.xml", "w") as f:
    f.write("<quiz>\n")
    for i, question_idx in enumerate(generated_questions, start=1):
        qcm = processed_data[question_idx] 
        f.write(f"  <!-- question: {i} -->\n")
        f.write(f"  <question type=\"multichoice\">\n")
        f.write(f"    <name>\n")
        f.write(f"      <text>Question {i}</text>\n")
        f.write(f"    </name>\n")
        f.write(f"    <questiontext format=\"html\">\n")
        f.write(f"      <text><![CDATA[<pre>{qcm[0]}</pre>]]></text>\n")
        f.write(f"    </questiontext>\n")
        f.write(f"    <generalfeedback format=\"html\">\n")
        f.write(f"      <text><![CDATA[<p>none</p>]]></text>\n")
        f.write(f"    </generalfeedback>\n")
        f.write(f"    <defaultgrade>1.0000000</defaultgrade>\n")
        f.write(f"    <penalty>0.3333333</penalty>\n")
        f.write(f"    <hidden>0</hidden>\n")
        f.write(f"    <single>true</single>\n")
        f.write(f"    <shuffleanswers>true</shuffleanswers>\n")
        f.write(f"    <answernumbering>abc</answernumbering>\n")
        for j, option in enumerate(qcm[1], start=1):
            option_text = option.argmax() + 1 
            f.write(f"    <answer fraction=\"{'100' if np.argmax(qcm[2]) == j - 1 else '0'}\" format=\"html\">\n")
            f.write(f"      <text><![CDATA[{option_text}. {qcm[1][j-1]}]]></text>\n")
            f.write(f"      <feedback format=\"html\">\n")
            f.write(f"        <text><![CDATA[<p>none</p>]]></text>\n")
            f.write(f"      </feedback>\n")
            f.write(f"    </answer>\n")
        f.write(f"  </question>\n")
        f.write(f"\n") #je retourne a la ligne pour chaque nouvelle question
    f.write("</quiz>\n")


Epoch 1/4
1/1 [==============================] - 3s 3s/step - loss: 1.3605
Epoch 2/4
1/1 [==============================] - 0s 11ms/step - loss: 0.9855
Epoch 3/4
1/1 [==============================] - 0s 14ms/step - loss: 0.9363
Epoch 4/4
1/1 [==============================] - 0s 463ms/step
{'question': "Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 'options': ["Parcours en profondeur d'abord (DFS)", "Parcours en largeur d'abord (BFS)", 'Algorithme de Kruskal', 'Algorithme de Prim'], 'correct_answer': "Parcours en profondeur d'abord (DFS)"}
1/1 [==============================] - 0s 27ms/step
{'question': 'Quelle est la complexité temporelle du tri fusion ?', 'options': ['O(n log n)', 'O(n^2)', 'O(n)', 'O(log n)'], 'correct_answer': 'O(n log n)'}
1/1 [==============================] - 0s 29ms/step
{'question': "Quel est l'algorithme de recherche le plus efficace ?", 'options': ['Recherche séquentielle', 'Recherche binaire', 'Recherche expone

In [34]:
########## Good mais pour le  moment le formatage dans le fichier .xml se fait avec les donneés d'origine ##########

import random
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quelle est la complexité temporelle de la recherche séquentielle ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n)"),
    ("Quelle est la complexité spatiale de l'algorithme de tri rapide ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Dijkstra"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche linéaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour trouver un arbre couvrant minimal dans un graphe pondéré ?", 
     ["Algorithme de Prim", "Algorithme de Kruskal", "Algorithme de Dijkstra", "Algorithme de Bellman-Ford"], 
     "Algorithme de Prim"),
    ("Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe avec des arêtes de poids négatif ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Bellman-Ford"),
    ("Quelle est la complexité temporelle de l'algorithme de Floyd-Warshall ?", 
     ["O(n^3)", "O(n^2)", "O(n log n)", "O(n)"], 
     "O(n^3)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 
     ["Parcours en profondeur d'abord (DFS)", "Parcours en largeur d'abord (BFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en profondeur d'abord (DFS)"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche en profondeur ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en largeur dans un graphe ?", 
     ["Parcours en largeur d'abord (BFS)", "Parcours en profondeur d'abord (DFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en largeur d'abord (BFS)")
]

def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(processed_data[0][1])))) 
model.add(Dense(len(processed_data[0][1]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)

def generate_question(model, generated_questions):
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions]
    question_idx = random.choice(unused_questions)
    generated_questions.add(question_idx)
    question_text, options, correct_answer = processed_data[question_idx]
    
    predicted_answer = model.predict(np.array([options]))
    predicted_answer_index = np.argmax(predicted_answer)
    predicted_answer_text = options[predicted_answer_index]
    
    options_text = [option for option in data[question_idx][1]]
    correct_answer_text = data[question_idx][2]
    
    return {"question": question_text, "options": options_text, "correct_answer": correct_answer_text}

generated_questions = set() 
for _ in range(10):
    question = generate_question(model, generated_questions)
    print(question)

with open("BANK_QUESTIONS_6.xml", "w") as f:
    f.write("<quiz>\n")
    for i, question_idx in enumerate(generated_questions, start=1):
        qcm = processed_data[question_idx]
        f.write(f"  <!-- question: {i} -->\n")
        f.write(f"  <question type=\"multichoice\">\n")
        f.write(f"    <name>\n")
        f.write(f"      <text>Question {i}</text>\n")
        f.write(f"    </name>\n")
        f.write(f"    <questiontext format=\"html\">\n")
        f.write(f"      <text><![CDATA[<pre>{qcm[0]}</pre>]]></text>\n")
        f.write(f"    </questiontext>\n")
        f.write(f"    <generalfeedback format=\"html\">\n")
        f.write(f"      <text><![CDATA[<p>none</p>]]></text>\n")
        f.write(f"    </generalfeedback>\n")
        f.write(f"    <defaultgrade>1.0000000</defaultgrade>\n")
        f.write(f"    <penalty>0.3333333</penalty>\n")
        f.write(f"    <hidden>0</hidden>\n")
        f.write(f"    <single>true</single>\n")
        f.write(f"    <shuffleanswers>true</shuffleanswers>\n")
        f.write(f"    <answernumbering>abc</answernumbering>\n")
        for j, option in enumerate(qcm[1], start=1):
            option_text = chr(96 + j)  # je Convertis l'indice en lettre (a, b, c, d, ...)
            f.write(f"    <answer fraction=\"{'100' if np.argmax(qcm[2]) == j - 1 else '0'}\" format=\"html\">\n")
            f.write(f"      <text><![CDATA[{option_text}. {data[question_idx][1][j-1]}]]></text>\n")  # Utiliser les données d'origine
            f.write(f"      <feedback format=\"html\">\n")
            f.write(f"        <text><![CDATA[<p>none</p>]]></text>\n")
            f.write(f"      </feedback>\n")
            f.write(f"    </answer>\n")
        f.write(f"  </question>\n")
        f.write(f"\n")
    f.write("</quiz>\n")



Epoch 1/4
1/1 [==============================] - 2s 2s/step - loss: 1.3853
Epoch 2/4
1/1 [==============================] - 0s 15ms/step - loss: 1.0506
Epoch 3/4
1/1 [==============================] - 0s 18ms/step - loss: 0.8851
Epoch 4/4
1/1 [==============================] - 1s 508ms/step
{'question': "Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 'options': ['O(n log n)', 'O(n)', 'O(log n)', 'O(n^2)'], 'correct_answer': 'O(n log n)'}
1/1 [==============================] - 0s 35ms/step
{'question': "Quel est l'algorithme de tri adapté aux listes de petite taille ?", 'options': ['Tri par insertion', 'Tri fusion', 'Tri rapide', 'Tri à bulles'], 'correct_answer': 'Tri par insertion'}
1/1 [==============================] - 0s 32ms/step
{'question': 'Quelle est la complexité temporelle de la recherche séquentielle ?', 'options': ['O(n log n)', 'O(n)', 'O(log n)', 'O(n^2)'], 'correct_answer': 'O(n)'}
1/1 [==============================] - 0s 28ms/step
{'question': "Que

In [45]:

#############  Final ####################

import random
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

data = [
    ("Quel est l'algorithme de tri le plus rapide ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri rapide"),
    ("Quelle est la complexité temporelle du tri fusion ?", 
     ["O(n log n)", "O(n^2)", "O(n)", "O(log n)"], 
     "O(n log n)"),
    ("Quel est l'algorithme de recherche le plus efficace ?", 
     ["Recherche séquentielle", "Recherche binaire", "Recherche exponentielle", "Recherche linéaire"], 
     "Recherche binaire"),
    ("Quelle est la complexité temporelle de la recherche binaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(log n)"),
    ("Quel est l'algorithme de tri adapté aux listes presque triées ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quel est l'algorithme de tri adapté aux listes de petite taille ?", 
     ["Tri par insertion", "Tri fusion", "Tri rapide", "Tri à bulles"], 
     "Tri par insertion"),
    ("Quelle est la complexité temporelle de la recherche séquentielle ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n)"),
    ("Quelle est la complexité spatiale de l'algorithme de tri rapide ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe pondéré ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Dijkstra"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche linéaire ?", 
     ["O(log n)", "O(n)", "O(n log n)", "O(n^2)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour trouver un arbre couvrant minimal dans un graphe pondéré ?", 
     ["Algorithme de Prim", "Algorithme de Kruskal", "Algorithme de Dijkstra", "Algorithme de Bellman-Ford"], 
     "Algorithme de Prim"),
    ("Quelle est la complexité temporelle de l'algorithme de Kruskal ?", 
     ["O(n log n)", "O(n)", "O(log n)", "O(n^2)"], 
     "O(n log n)"),
    ("Quel est l'algorithme utilisé pour trouver le plus court chemin dans un graphe avec des arêtes de poids négatif ?", 
     ["Algorithme de Dijkstra", "Algorithme de Bellman-Ford", "Algorithme de Floyd-Warshall", "Algorithme de Kruskal"], 
     "Algorithme de Bellman-Ford"),
    ("Quelle est la complexité temporelle de l'algorithme de Floyd-Warshall ?", 
     ["O(n^3)", "O(n^2)", "O(n log n)", "O(n)"], 
     "O(n^3)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en profondeur dans un graphe ?", 
     ["Parcours en profondeur d'abord (DFS)", "Parcours en largeur d'abord (BFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en profondeur d'abord (DFS)"),
    ("Quelle est la complexité temporelle de l'algorithme de recherche en profondeur ?", 
     ["O(n)", "O(log n)", "O(n^2)", "O(n log n)"], 
     "O(n)"),
    ("Quel est l'algorithme utilisé pour effectuer une recherche en largeur dans un graphe ?", 
     ["Parcours en largeur d'abord (BFS)", "Parcours en profondeur d'abord (DFS)", "Algorithme de Kruskal", "Algorithme de Prim"], 
     "Parcours en largeur d'abord (BFS)")
]

def preprocess_data(data):
    processed_data = []
    for question, options, answer in data:
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(processed_data[0][1]))))
model.add(Dense(len(processed_data[0][1]), activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01)) 

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)

def generate_question(model, generated_questions):
    # Générer une question aléatoire non utilisée
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions]
    if not unused_questions:  # Vérifier si la liste est vide
        return None  # Retourner None pour indiquer que toutes les questions ont été utilisées
    
    question_idx = random.choice(unused_questions)
    generated_questions.add(question_idx)
    question_text, options, correct_answer = processed_data[question_idx]
    
    predicted_answer = model.predict(np.array([options]))
    predicted_answer_index = np.argmax(predicted_answer)
    predicted_answer_text = options[predicted_answer_index]
    
    options_text = [option for option in data[question_idx][1]]
    correct_answer_text = data[question_idx][2]
    
    return {"question": question_text, "options": options_text, "correct_answer": correct_answer_text}

generated_questions = set() 
for _ in range(10):
    question = generate_question(model, generated_questions)
    if question:  # Vérifier si la question est None
        print(question)
    else:
        print("Toutes les questions ont été utilisées.")
        break  # Arrêter la génération de questions


with open("BANK_QUESTIONS_7.xml", "w") as f:
    f.write("<quiz>\n")
    generated_questions_copy = generated_questions.copy()  # Faire une copie de l'ensemble
    for i, question_idx in enumerate(generated_questions_copy, start=1):
        qcm = generate_question(model, generated_questions)  # Utiliser le modèle pour générer une question
        if qcm is None:  # Vérifier si aucune question n'a été générée
            print("Toutes les questions ont été utilisées.")
            break  # Sortir de la boucle
        f.write(f"  <!-- question: {i} -->\n")
        f.write(f"  <question type=\"multichoice\">\n")
        f.write(f"    <name>\n")
        f.write(f"      <text>Question {i}</text>\n")
        f.write(f"    </name>\n")
        f.write(f"    <questiontext format=\"html\">\n")
        f.write(f"      <text><![CDATA[<pre>{qcm['question']}</pre>]]></text>\n")
        f.write(f"    </questiontext>\n")
        f.write(f"    <generalfeedback format=\"html\">\n")
        f.write(f"      <text><![CDATA[<p>none</p>]]></text>\n")
        f.write(f"    </generalfeedback>\n")
        f.write(f"    <defaultgrade>1.0000000</defaultgrade>\n")
        f.write(f"    <penalty>0.3333333</penalty>\n")
        f.write(f"    <hidden>0</hidden>\n")
        f.write(f"    <single>true</single>\n")
        f.write(f"    <shuffleanswers>true</shuffleanswers>\n")
        f.write(f"    <answernumbering>abc</answernumbering>\n")
        for j, option in enumerate(qcm['options'], start=1):
            is_correct = (option == qcm['correct_answer'])  # Vérifier si l'option est la bonne réponse
            fraction = '100' if is_correct else '0'
            f.write(f"    <answer fraction=\"{fraction}\" format=\"html\">\n")
            f.write(f"      <text><![CDATA[{j}. {option}]]></text>\n")
            f.write(f"      <feedback format=\"html\">\n")
            f.write(f"        <text><![CDATA[<p>none</p>]]></text>\n")
            f.write(f"      </feedback>\n")
            f.write(f"    </answer>\n")
        f.write(f"  </question>\n")
    f.write("</quiz>\n")



Epoch 1/4
1/1 [==============================] - 4s 4s/step - loss: 1.3973
Epoch 2/4
1/1 [==============================] - 0s 41ms/step - loss: 1.0102
Epoch 3/4
1/1 [==============================] - 0s 30ms/step - loss: 0.9030
Epoch 4/4
1/1 [==============================] - 1s 568ms/step
{'question': "Quel est l'algorithme de recherche le plus efficace ?", 'options': ['Recherche séquentielle', 'Recherche binaire', 'Recherche exponentielle', 'Recherche linéaire'], 'correct_answer': 'Recherche binaire'}
1/1 [==============================] - 0s 26ms/step
{'question': "Quel est l'algorithme utilisé pour effectuer une recherche en largeur dans un graphe ?", 'options': ["Parcours en largeur d'abord (BFS)", "Parcours en profondeur d'abord (DFS)", 'Algorithme de Kruskal', 'Algorithme de Prim'], 'correct_answer': "Parcours en largeur d'abord (BFS)"}
1/1 [==============================] - 0s 31ms/step
{'question': "Quelle est la complexité temporelle de l'algorithme de recherche en profondeu

In [2]:
############### Final with data in my .JSON file #############

import random
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LSTM, Dense

# Charger les données depuis le fichier JSON
with open("data.json", "r") as json_file:
    data = json.load(json_file)["questions"]

# Prétraitement des données
def preprocess_data(data):
    processed_data = []
    for question_data in data:
        question = question_data["question"]
        options = question_data["options"]
        answer = question_data["answer"]
        options_one_hot = [tf.keras.utils.to_categorical(i, num_classes=len(options)) for i in range(len(options))]
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        processed_data.append((question, options_one_hot, answer_one_hot))
    return processed_data

processed_data = preprocess_data(data)

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(processed_data[0][1])))) 
model.add(Dense(len(processed_data[0][1]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

x_train = np.array([options for _, options, _ in processed_data])
y_train = np.array([answer for _, _, answer in processed_data])
model.fit(x_train, y_train, batch_size=256, epochs=4)

def generate_question(model, generated_questions):
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions]
    if not unused_questions:  
        return None  
    
    question_idx = random.choice(unused_questions)
    generated_questions.add(question_idx)
    question_text, options, correct_answer = processed_data[question_idx]
    
    predicted_answer = model.predict(np.array([options]))
    predicted_answer_index = np.argmax(predicted_answer)
    predicted_answer_text = options[predicted_answer_index]
    
    options_text = [option for option in data[question_idx]["options"]]
    correct_answer_text = data[question_idx]["answer"]
    
    return {"question": question_text, "options": options_text, "correct_answer": correct_answer_text}

generated_questions = set()
for _ in range(10):
    question = generate_question(model, generated_questions)
    if question: 
        print(question)
    else:
        print("Toutes les questions ont été utilisées.")
        break  


with open("BANK_QUESTIONS_8.xml", "w") as f:
    f.write("<quiz>\n")
    generated_questions_copy = generated_questions.copy() 
    for i, question_idx in enumerate(generated_questions_copy, start=1):
        qcm = generate_question(model, generated_questions) 
        if qcm is None:  # Vérifier si aucune question n'a été générée
            print("Toutes les questions ont été utilisées.")
            break  # Sortir de la boucle
        f.write(f"  <!-- question: {i} -->\n")
        f.write(f"  <question type=\"multichoice\">\n")
        f.write(f"    <name>\n")
        f.write(f"      <text>Question {i}</text>\n")
        f.write(f"    </name>\n")
        f.write(f"    <questiontext format=\"html\">\n")
        f.write(f"      <text><![CDATA[<pre>{qcm['question']}</pre>]]></text>\n")
        f.write(f"    </questiontext>\n")
        f.write(f"    <generalfeedback format=\"html\">\n")
        f.write(f"      <text><![CDATA[<p>none</p>]]></text>\n")
        f.write(f"    </generalfeedback>\n")
        f.write(f"    <defaultgrade>1.0000000</defaultgrade>\n")
        f.write(f"    <penalty>0.3333333</penalty>\n")
        f.write(f"    <hidden>0</hidden>\n")
        f.write(f"    <single>true</single>\n")
        f.write(f"    <shuffleanswers>true</shuffleanswers>\n")
        f.write(f"    <answernumbering>abc</answernumbering>\n")
        for j, option in enumerate(qcm['options'], start=1):
            is_correct = (option == qcm['correct_answer'])  # Vérifier si l'option est la bonne réponse
            fraction = '100' if is_correct else '0'
            f.write(f"    <answer fraction=\"{fraction}\" format=\"html\">\n")
            f.write(f"      <text><![CDATA[{j}. {option}]]></text>\n")
            f.write(f"      <feedback format=\"html\">\n")
            f.write(f"        <text><![CDATA[<p>none</p>]]></text>\n")
            f.write(f"      </feedback>\n")
            f.write(f"    </answer>\n")
        f.write(f"  </question>\n")
    f.write("</quiz>\n")


Epoch 1/4
1/1 [==============================] - 2s 2s/step - loss: 1.3965
Epoch 2/4
1/1 [==============================] - 0s 12ms/step - loss: 1.2577
Epoch 3/4
1/1 [==============================] - 0s 14ms/step - loss: 1.2702
Epoch 4/4
1/1 [==============================] - 0s 451ms/step
{'question': "Quel est l'algorithme de tri adaptÃ© aux listes de petite taille ?", 'options': ['Tri par insertion', 'Tri fusion', 'Tri rapide', 'Tri Ã\xa0 bulles'], 'correct_answer': 'Tri par insertion'}
1/1 [==============================] - 0s 25ms/step
{'question': "RÃ‰PÃ‰TER <traitement> JUSQU'Ã€ <condition> est une ______?", 'options': ['Boucle positive', 'Les deux', 'Boucle nÃ©gative', 'Aucun'], 'correct_answer': 'Boucle positive'}
1/1 [==============================] - 0s 26ms/step
{'question': "Combien de comparaisons effectuera l'algorithme de tri par insertion si on l'applique Ã\xa0 un tableau de n Ã©lÃ©ments dÃ©jÃ\xa0 triÃ© ?", 'options': ['(nÂ²+n)/2', 'n-1', 'n.(n+1)', 'log(n)'], 'correc

In [1]:
################# Final : Approche Supervisé Encodage - One -Hot aussi de la question pas seulement des propositions et du label ########################

import random
import numpy as np
import tensorflow as tf
import json

with open("data.json", "r") as json_file:
    data = json.load(json_file)["questions"]

def preprocess_data(data):
    processed_data = []
    vocab = set()  # Créer un ensemble pour stocker toutes les lettres ou mots uniques
    
    # Parcourir les données pour construire le vocabulaire
    for question_data in data:
        question = question_data["question"]
        options = question_data["options"]
        answer = question_data["answer"]
        vocab.update(question)  # Ajouter toutes les lettres ou mots de la question au vocabulaire
        for option in options:
            vocab.update(option)  # Ajouter toutes les lettres ou mots des options au vocabulaire
    
    # Convertir le vocabulaire en une liste triée
    vocab = sorted(vocab)
    
    # Créer un dictionnaire pour mapper chaque caractère ou mot à un index
    vocab_mapping = {char: idx for idx, char in enumerate(vocab)}
    
    # Encoder les données
    for question_data in data:
        question = question_data["question"]
        options = question_data["options"]
        answer = question_data["answer"]
        
        # Encoder la question avec one-hot encoding
        question_one_hot = [vocab_mapping[char] for char in question]
        question_one_hot = tf.keras.utils.to_categorical(question_one_hot, num_classes=len(vocab))
        
        # Padding des options pour avoir la même longueur
        max_option_length = max(len(option) for option in options)
        padded_options = [[vocab_mapping[char] for char in option] + [0] * (max_option_length - len(option)) for option in options]
        options_one_hot = np.array(padded_options)  # Convertir en tableau NumPy
        
        # Encoder la réponse avec one-hot encoding
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        
        processed_data.append((question_one_hot, options_one_hot, answer_one_hot))
    
    return processed_data, vocab_mapping

processed_data, vocab_mapping = preprocess_data(data)

# Modifier le modèle pour traiter les données encodées avec one-hot encoding
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(None, len(vocab_mapping))))  # Utiliser la longueur du vocabulaire comme entrée
model.add(tf.keras.layers.Dense(len(processed_data[0][1]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01))

# Entraînement du modèle
max_option_length = max(options.shape[1] for _, options, _ in processed_data)
x_train = np.array([np.pad(options, ((0, 0), (0, max_option_length - options.shape[1])), 'constant', constant_values=0) for _, options, _ in processed_data])
# Vérifier les dimensions des données d'entrée
print("Shape of x_train before:", x_train.shape)

# Supprimer la dimension inutile
x_train = x_train[:, :, :81]  # Prendre uniquement les 81 premières colonnes
print("Shape of x_train after:", x_train.shape)

y_train = np.array([answer for _, _, answer in processed_data])

model.fit(x_train, y_train, batch_size=256, epochs=4)


# Fonction pour générer une question
def generate_question(model, generated_questions):
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions]
    if not unused_questions:
        return None

    question_idx = random.choice(unused_questions)
    generated_questions.add(question_idx)
    question_one_hot, options, correct_answer = processed_data[question_idx]

    predicted_answer = model.predict(np.array([question_one_hot]))  # Utiliser la question encodée avec one-hot encoding
    predicted_answer_index = np.argmax(predicted_answer)
    predicted_answer_text = options[predicted_answer_index]

    options_text = [option for option in data[question_idx]["options"]]
    correct_answer_text = data[question_idx]["answer"]

    return {"question": data[question_idx]["question"], "options": options_text, "correct_answer": correct_answer_text}

# Générerons des questions
generated_questions = set()
for _ in range(10):
    question = generate_question(model, generated_questions)
    if question:
        print(question)
    else:
        print("Toutes les questions ont été utilisées.")
        break

# Put dans un fichier XML
with open("BANK_QUESTIONS_10_FIN.xml", "w") as f:
    f.write("<quiz>\n")
    generated_questions_copy = generated_questions.copy()
    for i, question_idx in enumerate(generated_questions_copy, start=1):
        qcm = generate_question(model, generated_questions)
        if qcm is None:  # Vérifier si aucune question n'a été générée
            print("Toutes les questions ont été utilisées.")
            break  # Sortir de la boucle
        f.write(f"  <!-- question: {i} -->\n")
        f.write(f"  <question type=\"multichoice\">\n")
        f.write(f"    <name>\n")
        f.write(f"      <text>Question {i}</text>\n")
        f.write(f"    </name>\n")
        f.write(f"    <questiontext format=\"html\">\n")
        f.write(f"      <text><![CDATA[<pre>{qcm['question']}</pre>]]></text>\n")
        f.write(f"    </questiontext>\n")
        f.write(f"    <generalfeedback format=\"html\">\n")
        f.write(f"      <text><![CDATA[<p>none</p>]]></text>\n")
        f.write(f"    </generalfeedback>\n")
        f.write(f"    <defaultgrade>1.0000000</defaultgrade>\n")
        f.write(f"    <penalty>0.3333333</penalty>\n")
        f.write(f"    <hidden>0</hidden>\n")
        f.write(f"    <single>true</single>\n")
        f.write(f"    <shuffleanswers>true</shuffleanswers>\n")
        f.write(f"    <answernumbering>abc</answernumbering>\n")
        for j, option in enumerate(qcm['options'], start=1):
            is_correct = (option == qcm['correct_answer'])  # je verifie si l'option est la bonne réponse
            fraction = '100' if is_correct else '0'
            f.write(f"    <answer fraction=\"{fraction}\" format=\"html\">\n")
            f.write(f"      <text><![CDATA[{j}. {option}]]></text>\n")
            f.write(f"      <feedback format=\"html\">\n")
            f.write(f"        <text><![CDATA[<p>none</p>]]></text>\n")
            f.write(f"      </feedback>\n")
            f.write(f"    </answer>\n")
        f.write(f"  </question>\n")
    f.write("</quiz>\n")


Shape of x_train before: (37, 4, 137)
Shape of x_train after: (37, 4, 81)
Epoch 1/4
1/1 [==============================] - 7s 7s/step - loss: 1.4302
Epoch 2/4
1/1 [==============================] - 0s 13ms/step - loss: 1.2805
Epoch 3/4
1/1 [==============================] - 0s 13ms/step - loss: 1.2686
Epoch 4/4
1/1 [==============================] - 1s 633ms/step
{'question': 'Un arbre binaire est :', 'options': ['un algorithme de tri', 'un algorithme de parcourt de graphe', 'un algorithme de recherche', 'Aucune de ces propositions'], 'correct_answer': 'Aucune de ces propositions'}
1/1 [==============================] - 0s 471ms/step
{'question': 'Quelle est la complexitÃ© temporelle de la recherche sÃ©quentielle ?', 'options': ['O(n log n)', 'O(n)', 'O(log n)', 'O(n^2)'], 'correct_answer': 'O(n)'}
1/1 [==============================] - 0s 30ms/step
{'question': "Dans un organigramme, une instruction d'entrÃ©e ou de sortie est reprÃ©sentÃ©e par _____?", 'options': ['Un losange', 'Un re

In [4]:
############ Test Sans Encodage du Marks ##########################################
############################ Question One Marks ############

import random
import numpy as np
import tensorflow as tf
import json

# Charger les données depuis le fichier JSON
with open("data_marks.json", "r") as json_file:
    data = json.load(json_file)["questions"]

def preprocess_data(data):
    processed_data = []
    vocab = set()  # Créer un ensemble pour stocker toutes les lettres ou mots uniques
    
    # Parcourir les données pour construire le vocabulaire
    for question_data in data:
        question = question_data["question"]
        options = question_data["options"]
        answer = question_data["answer"]
        vocab.update(question)  # Ajouter toutes les lettres ou mots de la question au vocabulaire
        for option in options:
            vocab.update(option)  # Ajouter toutes les lettres ou mots des options au vocabulaire
    
    # Convertir le vocabulaire en une liste triée
    vocab = sorted(vocab)
    
    # Créer un dictionnaire pour mapper chaque caractère ou mot à un index
    vocab_mapping = {char: idx for idx, char in enumerate(vocab)}
    
    # Encoder les données
    for question_data in data:
        question = question_data["question"]
        options = question_data["options"]
        answer = question_data["answer"]
        
        # Encoder la question avec one-hot encoding
        question_one_hot = [vocab_mapping[char] for char in question]
        question_one_hot = tf.keras.utils.to_categorical(question_one_hot, num_classes=len(vocab))
        
        # Padding des options pour avoir la même longueur
        max_option_length = max(len(option) for option in options)
        padded_options = [[vocab_mapping[char] for char in option] + [0] * (max_option_length - len(option)) for option in options]
        options_one_hot = np.array(padded_options)  # Convertir en tableau NumPy
        
        # Encoder la réponse avec one-hot encoding
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        
        processed_data.append((question_one_hot, options_one_hot, answer_one_hot))
    
    return processed_data, vocab_mapping

processed_data, vocab_mapping = preprocess_data(data)

# Modifier le modèle pour traiter les données encodées avec one-hot encoding
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(None, len(vocab_mapping))))  # Utiliser la longueur du vocabulaire comme entrée
model.add(tf.keras.layers.Dense(len(processed_data[0][1]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01))

# Entraînement du modèle
max_option_length = max(options.shape[1] for _, options, _ in processed_data)
x_train = np.array([np.pad(options, ((0, 0), (0, max_option_length - options.shape[1])), 'constant', constant_values=0) for _, options, _ in processed_data])
# Vérifier les dimensions des données d'entrée
print("Shape of x_train before:", x_train.shape)

# Supprimer la dimension inutile
x_train = x_train[:, :, :81]  # Prendre uniquement les 81 premières colonnes
print("Shape of x_train after:", x_train.shape)

y_train = np.array([answer for _, _, answer in processed_data])

model.fit(x_train, y_train, batch_size=256, epochs=4)


# Fonction pour générer une question
def generate_question(model, generated_questions, marks, max_questions):
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions and data[idx]["marks"] == marks]
    if not unused_questions:
        return None

    if len(unused_questions) > max_questions - len(generated_questions):
        unused_questions = random.sample(unused_questions, max_questions - len(generated_questions))

    for question_idx in unused_questions:
        generated_questions.add(question_idx)
        question_one_hot, options, correct_answer = processed_data[question_idx]

        predicted_answer = model.predict(np.array([question_one_hot]))  # Utiliser la question encodée avec one-hot encoding
        predicted_answer_index = np.argmax(predicted_answer)
        predicted_answer_text = options[predicted_answer_index]

        options_text = [option for option in data[question_idx]["options"]]
        correct_answer_text = data[question_idx]["answer"]

        yield {"question": data[question_idx]["question"], "options": options_text, "correct_answer": correct_answer_text, "marks": marks}


# Générer des questions avec les paramètres spécifiés
def generate_questions_to_xml(model, marks, nbre_questions, output_file):
    generated_questions = set()
    with open(output_file, "w") as f:
        f.write("<quiz>\n")
        for i, qcm in enumerate(generate_question(model, generated_questions, marks, nbre_questions), start=1):
            f.write(f"  <!-- question: {i} -->\n")
            f.write(f"  <question type=\"multichoice\">\n")
            f.write(f"    <name>\n")
            f.write(f"      <text>Question {i}</text>\n")
            f.write(f"    </name>\n")
            f.write(f"    <questiontext format=\"html\">\n")
            f.write(f"      <text><![CDATA[<pre>{qcm['question']}</pre>]]></text>\n")
            f.write(f"    </questiontext>\n")
            f.write(f"    <generalfeedback format=\"html\">\n")
            f.write(f"      <text><![CDATA[<p>none</p>]]></text>\n")
            f.write(f"    </generalfeedback>\n")
            f.write(f"    <defaultgrade>{qcm['marks']}.0000000</defaultgrade>\n")
            f.write(f"    <penalty>0.3333333</penalty>\n")
            f.write(f"    <hidden>0</hidden>\n")
            f.write(f"    <single>true</single>\n")
            f.write(f"    <shuffleanswers>true</shuffleanswers>\n")
            f.write(f"    <answernumbering>abc</answernumbering>\n")
            for j, option in enumerate(qcm['options'], start=1):
                is_correct = (option == qcm['correct_answer'])  # je verifie si l'option est la bonne réponse
                fraction = '100' if is_correct else '0'
                f.write(f"    <answer fraction=\"{fraction}\" format=\"html\">\n")
                f.write(f"      <text><![CDATA[{j}. {option}]]></text>\n")
                f.write(f"      <feedback format=\"html\">\n")
                f.write(f"        <text><![CDATA[<p>none</p>]]></text>\n")
                f.write(f"      </feedback>\n")
                f.write(f"    </answer>\n")
            f.write(f"  </question>\n")
        f.write("</quiz>\n")

# Exemple d'utilisation
generate_questions_to_xml(model, marks=1, nbre_questions=3, output_file="marks_1.xml")



Shape of x_train before: (37, 4, 137)
Shape of x_train after: (37, 4, 81)
Epoch 1/4
1/1 [==============================] - 2s 2s/step - loss: 1.3936
Epoch 2/4
1/1 [==============================] - 0s 10ms/step - loss: 1.2180
Epoch 3/4
1/1 [==============================] - 0s 13ms/step - loss: 1.5195
Epoch 4/4
1/1 [==============================] - 0s 31ms/step


In [5]:
############################ Question Two Marks ############


############ Test ##########################################
############################ Question One Marks ############

import random
import numpy as np
import tensorflow as tf
import json

# Charger les données depuis le fichier JSON
with open("data_marks.json", "r") as json_file:
    data = json.load(json_file)["questions"]

def preprocess_data(data):
    processed_data = []
    vocab = set()  # Créer un ensemble pour stocker toutes les lettres ou mots uniques
    
    # Parcourir les données pour construire le vocabulaire
    for question_data in data:
        question = question_data["question"]
        options = question_data["options"]
        answer = question_data["answer"]
        vocab.update(question)  # Ajouter toutes les lettres ou mots de la question au vocabulaire
        for option in options:
            vocab.update(option)  # Ajouter toutes les lettres ou mots des options au vocabulaire
    
    # Convertir le vocabulaire en une liste triée
    vocab = sorted(vocab)
    
    # Créer un dictionnaire pour mapper chaque caractère ou mot à un index
    vocab_mapping = {char: idx for idx, char in enumerate(vocab)}
    
    # Encoder les données
    for question_data in data:
        question = question_data["question"]
        options = question_data["options"]
        answer = question_data["answer"]
        
        # Encoder la question avec one-hot encoding
        question_one_hot = [vocab_mapping[char] for char in question]
        question_one_hot = tf.keras.utils.to_categorical(question_one_hot, num_classes=len(vocab))
        
        # Padding des options pour avoir la même longueur
        max_option_length = max(len(option) for option in options)
        padded_options = [[vocab_mapping[char] for char in option] + [0] * (max_option_length - len(option)) for option in options]
        options_one_hot = np.array(padded_options)  # Convertir en tableau NumPy
        
        # Encoder la réponse avec one-hot encoding
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        
        processed_data.append((question_one_hot, options_one_hot, answer_one_hot))
    
    return processed_data, vocab_mapping

processed_data, vocab_mapping = preprocess_data(data)

# Modifier le modèle pour traiter les données encodées avec one-hot encoding
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(None, len(vocab_mapping))))  # Utiliser la longueur du vocabulaire comme entrée
model.add(tf.keras.layers.Dense(len(processed_data[0][1]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01))

# Entraînement du modèle
max_option_length = max(options.shape[1] for _, options, _ in processed_data)
x_train = np.array([np.pad(options, ((0, 0), (0, max_option_length - options.shape[1])), 'constant', constant_values=0) for _, options, _ in processed_data])
# Vérifier les dimensions des données d'entrée
print("Shape of x_train before:", x_train.shape)

# Supprimer la dimension inutile
x_train = x_train[:, :, :81]  # Prendre uniquement les 81 premières colonnes
print("Shape of x_train after:", x_train.shape)

y_train = np.array([answer for _, _, answer in processed_data])

model.fit(x_train, y_train, batch_size=256, epochs=4)


# Fonction pour générer une question
def generate_question(model, generated_questions, marks, max_questions):
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions and data[idx]["marks"] == marks]
    if not unused_questions:
        return None

    if len(unused_questions) > max_questions - len(generated_questions):
        unused_questions = random.sample(unused_questions, max_questions - len(generated_questions))

    for question_idx in unused_questions:
        generated_questions.add(question_idx)
        question_one_hot, options, correct_answer = processed_data[question_idx]

        predicted_answer = model.predict(np.array([question_one_hot]))  # Utiliser la question encodée avec one-hot encoding
        predicted_answer_index = np.argmax(predicted_answer)
        predicted_answer_text = options[predicted_answer_index]

        options_text = [option for option in data[question_idx]["options"]]
        correct_answer_text = data[question_idx]["answer"]

        yield {"question": data[question_idx]["question"], "options": options_text, "correct_answer": correct_answer_text, "marks": marks}


# Générer des questions avec les paramètres spécifiés
def generate_questions_to_xml(model, marks, nbre_questions, output_file):
    generated_questions = set()
    with open(output_file, "w") as f:
        f.write("<quiz>\n")
        for i, qcm in enumerate(generate_question(model, generated_questions, marks, nbre_questions), start=1):
            f.write(f"  <!-- question: {i} -->\n")
            f.write(f"  <question type=\"multichoice\">\n")
            f.write(f"    <name>\n")
            f.write(f"      <text>Question {i}</text>\n")
            f.write(f"    </name>\n")
            f.write(f"    <questiontext format=\"html\">\n")
            f.write(f"      <text><![CDATA[<pre>{qcm['question']}</pre>]]></text>\n")
            f.write(f"    </questiontext>\n")
            f.write(f"    <generalfeedback format=\"html\">\n")
            f.write(f"      <text><![CDATA[<p>none</p>]]></text>\n")
            f.write(f"    </generalfeedback>\n")
            f.write(f"    <defaultgrade>{qcm['marks']}.0000000</defaultgrade>\n")
            f.write(f"    <penalty>0.3333333</penalty>\n")
            f.write(f"    <hidden>0</hidden>\n")
            f.write(f"    <single>true</single>\n")
            f.write(f"    <shuffleanswers>true</shuffleanswers>\n")
            f.write(f"    <answernumbering>abc</answernumbering>\n")
            for j, option in enumerate(qcm['options'], start=1):
                is_correct = (option == qcm['correct_answer'])  # je verifie si l'option est la bonne réponse
                fraction = '100' if is_correct else '0'
                f.write(f"    <answer fraction=\"{fraction}\" format=\"html\">\n")
                f.write(f"      <text><![CDATA[{j}. {option}]]></text>\n")
                f.write(f"      <feedback format=\"html\">\n")
                f.write(f"        <text><![CDATA[<p>none</p>]]></text>\n")
                f.write(f"      </feedback>\n")
                f.write(f"    </answer>\n")
            f.write(f"  </question>\n")
        f.write("</quiz>\n")

# Exemple d'utilisation
generate_questions_to_xml(model, marks=2, nbre_questions=3, output_file="marks_2.xml")



Shape of x_train before: (37, 4, 137)
Shape of x_train after: (37, 4, 81)
Epoch 1/4
1/1 [==============================] - 3s 3s/step - loss: 1.3494
Epoch 2/4
1/1 [==============================] - 0s 17ms/step - loss: 1.1113
Epoch 3/4
1/1 [==============================] - 0s 15ms/step - loss: 1.4775
Epoch 4/4
1/1 [==============================] - 0s 38ms/step


In [6]:
############################ Question Three Marks ############

############ Test ##########################################
############################ Question One Marks ############

import random
import numpy as np
import tensorflow as tf
import json

# Charger les données depuis le fichier JSON
with open("data_marks.json", "r") as json_file:
    data = json.load(json_file)["questions"]

def preprocess_data(data):
    processed_data = []
    vocab = set()  # Créer un ensemble pour stocker toutes les lettres ou mots uniques
    
    # Parcourir les données pour construire le vocabulaire
    for question_data in data:
        question = question_data["question"]
        options = question_data["options"]
        answer = question_data["answer"]
        vocab.update(question)  # Ajouter toutes les lettres ou mots de la question au vocabulaire
        for option in options:
            vocab.update(option)  # Ajouter toutes les lettres ou mots des options au vocabulaire
    
    # Convertir le vocabulaire en une liste triée
    vocab = sorted(vocab)
    
    # Créer un dictionnaire pour mapper chaque caractère ou mot à un index
    vocab_mapping = {char: idx for idx, char in enumerate(vocab)}
    
    # Encoder les données
    for question_data in data:
        question = question_data["question"]
        options = question_data["options"]
        answer = question_data["answer"]
        
        # Encoder la question avec one-hot encoding
        question_one_hot = [vocab_mapping[char] for char in question]
        question_one_hot = tf.keras.utils.to_categorical(question_one_hot, num_classes=len(vocab))
        
        # Padding des options pour avoir la même longueur
        max_option_length = max(len(option) for option in options)
        padded_options = [[vocab_mapping[char] for char in option] + [0] * (max_option_length - len(option)) for option in options]
        options_one_hot = np.array(padded_options)  # Convertir en tableau NumPy
        
        # Encoder la réponse avec one-hot encoding
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        
        processed_data.append((question_one_hot, options_one_hot, answer_one_hot))
    
    return processed_data, vocab_mapping

processed_data, vocab_mapping = preprocess_data(data)

# Modifier le modèle pour traiter les données encodées avec one-hot encoding
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(None, len(vocab_mapping))))  # Utiliser la longueur du vocabulaire comme entrée
model.add(tf.keras.layers.Dense(len(processed_data[0][1]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01))

# Entraînement du modèle
max_option_length = max(options.shape[1] for _, options, _ in processed_data)
x_train = np.array([np.pad(options, ((0, 0), (0, max_option_length - options.shape[1])), 'constant', constant_values=0) for _, options, _ in processed_data])
# Vérifier les dimensions des données d'entrée
print("Shape of x_train before:", x_train.shape)

# Supprimer la dimension inutile
x_train = x_train[:, :, :81]  # Prendre uniquement les 81 premières colonnes
print("Shape of x_train after:", x_train.shape)

y_train = np.array([answer for _, _, answer in processed_data])

model.fit(x_train, y_train, batch_size=256, epochs=4)


# Fonction pour générer une question
def generate_question(model, generated_questions, marks, max_questions):
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions and data[idx]["marks"] == marks]
    if not unused_questions:
        return None

    if len(unused_questions) > max_questions - len(generated_questions):
        unused_questions = random.sample(unused_questions, max_questions - len(generated_questions))

    for question_idx in unused_questions:
        generated_questions.add(question_idx)
        question_one_hot, options, correct_answer = processed_data[question_idx]

        predicted_answer = model.predict(np.array([question_one_hot]))  # Utiliser la question encodée avec one-hot encoding
        predicted_answer_index = np.argmax(predicted_answer)
        predicted_answer_text = options[predicted_answer_index]

        options_text = [option for option in data[question_idx]["options"]]
        correct_answer_text = data[question_idx]["answer"]

        yield {"question": data[question_idx]["question"], "options": options_text, "correct_answer": correct_answer_text, "marks": marks}


# Générer des questions avec les paramètres spécifiés
def generate_questions_to_xml(model, marks, nbre_questions, output_file):
    generated_questions = set()
    with open(output_file, "w") as f:
        f.write("<quiz>\n")
        for i, qcm in enumerate(generate_question(model, generated_questions, marks, nbre_questions), start=1):
            f.write(f"  <!-- question: {i} -->\n")
            f.write(f"  <question type=\"multichoice\">\n")
            f.write(f"    <name>\n")
            f.write(f"      <text>Question {i}</text>\n")
            f.write(f"    </name>\n")
            f.write(f"    <questiontext format=\"html\">\n")
            f.write(f"      <text><![CDATA[<pre>{qcm['question']}</pre>]]></text>\n")
            f.write(f"    </questiontext>\n")
            f.write(f"    <generalfeedback format=\"html\">\n")
            f.write(f"      <text><![CDATA[<p>none</p>]]></text>\n")
            f.write(f"    </generalfeedback>\n")
            f.write(f"    <defaultgrade>{qcm['marks']}.0000000</defaultgrade>\n")
            f.write(f"    <penalty>0.3333333</penalty>\n")
            f.write(f"    <hidden>0</hidden>\n")
            f.write(f"    <single>true</single>\n")
            f.write(f"    <shuffleanswers>true</shuffleanswers>\n")
            f.write(f"    <answernumbering>abc</answernumbering>\n")
            for j, option in enumerate(qcm['options'], start=1):
                is_correct = (option == qcm['correct_answer'])  # je verifie si l'option est la bonne réponse
                fraction = '100' if is_correct else '0'
                f.write(f"    <answer fraction=\"{fraction}\" format=\"html\">\n")
                f.write(f"      <text><![CDATA[{j}. {option}]]></text>\n")
                f.write(f"      <feedback format=\"html\">\n")
                f.write(f"        <text><![CDATA[<p>none</p>]]></text>\n")
                f.write(f"      </feedback>\n")
                f.write(f"    </answer>\n")
            f.write(f"  </question>\n")
        f.write("</quiz>\n")

# Exemple d'utilisation
generate_questions_to_xml(model, marks=3, nbre_questions=3, output_file="marks_3.xml")




Shape of x_train before: (37, 4, 137)
Shape of x_train after: (37, 4, 81)
Epoch 1/4
1/1 [==============================] - 3s 3s/step - loss: 1.3485
Epoch 2/4
1/1 [==============================] - 0s 20ms/step - loss: 1.4009
Epoch 3/4
1/1 [==============================] - 0s 19ms/step - loss: 1.1216
Epoch 4/4
1/1 [==============================] - 0s 31ms/step


In [9]:
################## Avec encodage du Makrs ###################

import random
import numpy as np
import tensorflow as tf
import json

# Charger les données depuis le fichier JSON
with open("data_marks.json", "r") as json_file:
    data = json.load(json_file)["questions"]

def preprocess_data(data):
    processed_data = []
    vocab = set()  # Créer un ensemble pour stocker toutes les lettres ou mots uniques
    
    # Parcourir les données pour construire le vocabulaire
    for question_data in data:
        question = question_data["question"]
        options = question_data["options"]
        answer = question_data["answer"]
        marks = question_data.get("marks", 1)  # Utiliser 1 comme valeur par défaut si "marks" n'est pas présent
        vocab.update(question)  # Ajouter toutes les lettres ou mots de la question au vocabulaire
        for option in options:
            vocab.update(option)  # Ajouter toutes les lettres ou mots des options au vocabulaire
    
    # Convertir le vocabulaire en une liste triée
    vocab = sorted(vocab)
    
    # Créer un dictionnaire pour mapper chaque caractère ou mot à un index
    vocab_mapping = {char: idx for idx, char in enumerate(vocab)}
    
    # Encoder les données
    for question_data in data:
        question = question_data["question"]
        options = question_data["options"]
        answer = question_data["answer"]
        marks = question_data.get("marks", 1)  # Utiliser 1 comme valeur par défaut si "marks" n'est pas présent
        
        # Encoder la question avec one-hot encoding
        question_one_hot = [vocab_mapping[char] for char in question]
        question_one_hot = tf.keras.utils.to_categorical(question_one_hot, num_classes=len(vocab))
        
        # Padding des options pour avoir la même longueur
        max_option_length = max(len(option) for option in options)
        padded_options = [[vocab_mapping[char] for char in option] + [0] * (max_option_length - len(option)) for option in options]
        options_one_hot = np.array(padded_options)  # Convertir en tableau NumPy
        
        # Encoder la réponse avec one-hot encoding
        answer_index = options.index(answer)
        answer_one_hot = tf.keras.utils.to_categorical(answer_index, num_classes=len(options))
        
        processed_data.append((question_one_hot, options_one_hot, answer_one_hot, marks))
    
    return processed_data, vocab_mapping


processed_data, vocab_mapping = preprocess_data(data)

# Modifier le modèle pour traiter les données encodées avec one-hot encoding
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(None, len(vocab_mapping))))  # Utiliser la longueur du vocabulaire comme entrée
model.add(tf.keras.layers.Dense(len(processed_data[0][1]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01))

# Entraînement du modèle
max_option_length = max(options.shape[1] for _, options, _,_ in processed_data)
x_train = np.array([np.pad(options, ((0, 0), (0, max_option_length - options.shape[1])), 'constant', constant_values=0) for _, options, _, _ in processed_data])
# Vérifier les dimensions des données d'entrée
print("Shape of x_train before:", x_train.shape)

# Supprimer la dimension inutile
x_train = x_train[:, :, :81]  # Prendre uniquement les 81 premières colonnes
print("Shape of x_train after:", x_train.shape)

y_train = np.array([answer for _, _, answer, _ in processed_data])

model.fit(x_train, y_train, batch_size=256, epochs=4)


# Fonction pour générer une question
def generate_question(model, generated_questions, marks, max_questions):
    unused_questions = [idx for idx in range(len(data)) if idx not in generated_questions and data[idx]["marks"] == marks]
    if not unused_questions:
        return None

    if len(unused_questions) > max_questions - len(generated_questions):
        unused_questions = random.sample(unused_questions, max_questions - len(generated_questions))

    for question_idx in unused_questions:
        generated_questions.add(question_idx)
        question_one_hot, options, correct_answer, _ = processed_data[question_idx]

        predicted_answer = model.predict(np.array([question_one_hot]))  # Utiliser la question encodée avec one-hot encoding
        predicted_answer_index = np.argmax(predicted_answer)
        predicted_answer_text = options[predicted_answer_index]

        options_text = [option for option in data[question_idx]["options"]]
        correct_answer_text = data[question_idx]["answer"]

        yield {"question": data[question_idx]["question"], "options": options_text, "correct_answer": correct_answer_text, "marks": marks}


# Générer des questions avec les paramètres spécifiés
def generate_questions_to_xml(model, marks, nbre_questions, output_file):
    generated_questions = set()
    with open(output_file, "w") as f:
        f.write("<quiz>\n")
        for i, qcm in enumerate(generate_question(model, generated_questions, marks, nbre_questions), start=1):
            f.write(f"  <!-- question: {i} -->\n")
            f.write(f"  <question type=\"multichoice\">\n")
            f.write(f"    <name>\n")
            f.write(f"      <text>Question {i}</text>\n")
            f.write(f"    </name>\n")
            f.write(f"    <questiontext format=\"html\">\n")
            f.write(f"      <text><![CDATA[<pre>{qcm['question']}</pre>]]></text>\n")
            f.write(f"    </questiontext>\n")
            f.write(f"    <generalfeedback format=\"html\">\n")
            f.write(f"      <text><![CDATA[<p>none</p>]]></text>\n")
            f.write(f"    </generalfeedback>\n")
            f.write(f"    <defaultgrade>{qcm['marks']}.0000000</defaultgrade>\n")
            f.write(f"    <penalty>0.3333333</penalty>\n")
            f.write(f"    <hidden>0</hidden>\n")
            f.write(f"    <single>true</single>\n")
            f.write(f"    <shuffleanswers>true</shuffleanswers>\n")
            f.write(f"    <answernumbering>abc</answernumbering>\n")
            for j, option in enumerate(qcm['options'], start=1):
                is_correct = (option == qcm['correct_answer'])  # je verifie si l'option est la bonne réponse
                fraction = '100' if is_correct else '0'
                f.write(f"    <answer fraction=\"{fraction}\" format=\"html\">\n")
                f.write(f"      <text><![CDATA[{j}. {option}]]></text>\n")
                f.write(f"      <feedback format=\"html\">\n")
                f.write(f"        <text><![CDATA[<p>none</p>]]></text>\n")
                f.write(f"      </feedback>\n")
                f.write(f"    </answer>\n")
            f.write(f"  </question>\n")
        f.write("</quiz>\n")

# Exemple d'utilisation
generate_questions_to_xml(model, marks=3, nbre_questions=3, output_file="marks__encode.xml")


Shape of x_train before: (37, 4, 137)
Shape of x_train after: (37, 4, 81)
Epoch 1/4
1/1 [==============================] - 4s 4s/step - loss: 1.5715
Epoch 2/4
1/1 [==============================] - 0s 15ms/step - loss: 1.1785
Epoch 3/4
1/1 [==============================] - 0s 18ms/step - loss: 1.1905
Epoch 4/4
1/1 [==============================] - 0s 32ms/step
